# DADOS DE CARGA DE ENERGIA DIÁRIA

**Descrição do Dado:** Dados de carga por subsistema numa data de referência em base diária.

| Descrição | Código | Tipo de Dado | Formato | Permite valor nulo | Permite valor zerado | Permite valor negativo |
| --- | --- | --- | --- | --- | --- | --- |
| Código do Subsistema | `id_subsistema` | STRING | 3 POSIÇÕES | Não | - | - |
| Nome do Subsistema | `nom_subsistema` | STRING | 60 POSIÇÕES | Não | - | - |
| Data de referência | `din_instante` | DATETIME | `YYY-MM-DD HH:MM:SS` | Não | - | - |
| Valor da Carga de Energia (MWmed) | `val_cargaenergiamwmed` | FLOAT | - | Não | Não | Não |

<small>Última edição em `01-05-2023`.</small>

In [2]:
import pandas

In [3]:
carga_df = pandas.read_csv("CARGA_ENERGIA_2000.csv", sep=';')

In [4]:
carga_df.head(4)

,id_subsistema,nom_subsistema,din_instante,val_cargaenergiamwmed
0,N,Norte,2000-01-01,2243.512500
1,NE,Nordeste,2000-01-01,4646.370833
2,S,Sul,2000-01-01,4800.650000
3,SE,Sudeste/Centro-Oeste,2000-01-01,19045.995833


In [5]:
subsistemas = carga_df['id_subsistema'].unique()
nome_subsistemas = carga_df['nom_subsistema'].unique()
cargas = []

In [6]:
for subsistema, nome_subsistema in zip(subsistemas, nome_subsistemas):
    cargas.append({
        'id': subsistema,
        'nome': nome_subsistema,
        'registros': [
            # "data": DATETIME -- 'din_instante'
            # "carga" FLOAT -- 'val_cargaenergiamwmed'
        ]
    })

In [7]:
datas = carga_df['din_instante'].unique()

In [8]:
for data in datas:
    df = carga_df.loc[carga_df['din_instante'] == data]
    for carga in cargas:
        carga_id, _, carga_registros = carga
        energia = float(df.loc[df['id_subsistema'] == carga[carga_id]]\
            ['val_cargaenergiamwmed'].iloc[0])
        carga[carga_registros].append({
            'data': data,
            'energia': energia
        })

```shell
  id_subsistema        nom_subsistema din_instante  val_cargaenergiamwmed
0             N                 Norte   2000-01-01            2243.512500
1            NE              Nordeste   2000-01-01            4646.370833
2             S                   Sul   2000-01-01            4800.650000
3            SE  Sudeste/Centro-Oeste   2000-01-01           19045.995833 <class 'pandas.core.frame.DataFrame'>
```

In [10]:
import json
print(json.dumps(cargas))

[{"id": "N", "nome": "Norte", "registros": [{"data": "2000-01-01", "energia": 2243.5125}, {"data": "2000-01-02", "energia": 2259.80833333}, {"data": "2000-01-03", "energia": 2407.42916667}, {"data": "2000-01-04", "energia": 2404.21666667}, {"data": "2000-01-05", "energia": 2439.65}, {"data": "2000-01-06", "energia": 2426.68333333}, {"data": "2000-01-07", "energia": 2430.90833333}, {"data": "2000-01-08", "energia": 2286.35}, {"data": "2000-01-09", "energia": 2242.66666667}, {"data": "2000-01-10", "energia": 2417.59583333}, {"data": "2000-01-11", "energia": 2416.76666667}, {"data": "2000-01-12", "energia": 2418.35833333}, {"data": "2000-01-13", "energia": 2428.52083333}, {"data": "2000-01-14", "energia": 2440.20416667}, {"data": "2000-01-15", "energia": 2352.90833333}, {"data": "2000-01-16", "energia": 2263.8375}, {"data": "2000-01-17", "energia": 2433.975}, {"data": "2000-01-18", "energia": 2479.86666667}, {"data": "2000-01-19", "energia": 2439.2375}, {"data": "2000-01-20", "energia": 2